## 机器学习进阶（S13）毕业项目 - 猫狗大战

## 项目：识别图片中是猫还是狗
---
* 此项目是kaggle平台上的一个比赛项目，最终的要求是提供一个模型来识别图片中的对象是猫还是狗。
* 这里会使用深度学习中的卷积神经网络来构建模型。
* 最总要求测试评分进入 kaggle Public Leaderboard 前10%。

## 步骤 1: 数据预处理

### 载入数据

### 处理异常图片

## 步骤 2: 模型搭建

## 步骤 3: 模型训练

## 步骤 4: 模型调参

## 步骤 5: 模型评估

### 预测测试集

## 步骤 6: 可视化